In [2]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta
import mlxtend
# 판다스 엑셀 익스포트를 위한 라이브러리
import openpyxl


PROJECT = "ballosodeuk"
bq = bigquery.Client(project=PROJECT)


query = """
select * from ballosodeuk.ynam.rfm_table_byshop_category_power
"""

query = """
select user_id, depth1_list from ballosodeuk.dm.agg_user_category_rank
"""

df = bq.query(query).to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [3]:
df.head()

,user_id,depth1_list
0,74279df9-532d-4c40-ad88-9258f34cf58c,"식품, 생활-건강"
1,33223139-9a4e-4f09-87c8-c69d2fc14809,"식품, 패션의류"
2,58d70bf7-c0c7-4424-ad99-4b237753e5cb,식품
3,88f13bca-eff0-4ed8-9ea1-1c1bbaad5efb,"화장품-미용, 식품"
4,a0fa1c61-730b-4852-a1f0-a0d6d68c5a1c,식품


In [5]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns


# 2. 데이터 전처리 및 트랜잭션 인코딩
# depth1_list 열에서 쉼표로 구분된 아이템들을 리스트로 변환
df['depth1_list'] = df['depth1_list'].str.split(', ')

# 트랜잭션 리스트 생성
transactions = df['depth1_list'].tolist()
print(transactions)
# 3. 트랜잭션 인코딩 - 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Apriori 알고리즘으로 빈발 아이템셋 찾기
# min_support: 최소 지지도 (전체 거래 중 해당 아이템셋이 등장하는 비율)
# 작은 데이터셋이므로 낮은 지지도 사용
frequent_itemsets = apriori(df_encoded, min_support=000.1, use_colnames=True)
print("=== 빈발 아이템셋 ===")
print(frequent_itemsets)

# 5. 연관 규칙 생성
# min_threshold: 최소 신뢰도
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# 6. 결과 정렬 및 출력
# lift 기준으로 내림차순 정렬 (lift가 높을수록 강한 연관성)
rules_sorted = rules.sort_values('lift', ascending=False)
print("\n=== 연관 규칙 (lift 기준 정렬) ===")
print(rules_sorted)

# 7. 규칙 시각화 - lift 기준 상위 5개
def visualize_top_rules(rules, top_n=5, metric='lift'):
    """상위 규칙을 시각화하는 함수"""
    plt.figure(figsize=(10, 6))
    
    # 상위 n개 규칙 선택
    top_rules = rules.sort_values(metric, ascending=False).head(top_n)
    
    # 규칙 레이블 생성 (antecedent -> consequent)
    labels = [f"{', '.join(list(x))} → {', '.join(list(y))}" 
             for x, y in zip(top_rules['antecedents'], top_rules['consequents'])]
    
    # 바 차트 생성
    ax = sns.barplot(x=top_rules[metric], y=labels)
    
    # 그래프 레이블 및 제목
    plt.xlabel(f'{metric} 값')
    plt.ylabel('연관 규칙')
    plt.title(f'상위 {top_n}개 연관 규칙 ({metric} 기준)')
    
    # 바 끝에 값 표시
    for i, v in enumerate(top_rules[metric]):
        ax.text(v + 0.1, i, f'{v:.2f}', va='center')
    
    plt.tight_layout()
    return plt.gcf()  # 현재 figure 반환

# 상위 규칙 시각화
top_rules_fig = visualize_top_rules(rules_sorted)
top_rules_fig.savefig('top_rules.png')

# 8. 다양한 평가 지표로 규칙 분석
def analyze_rules_by_metrics(rules):
    """다양한 평가 지표로 규칙 분석"""
    # 신뢰도 vs 지지도
    plt.figure(figsize=(8, 6))
    plt.scatter(rules['support'], rules['confidence'], alpha=0.5)
    plt.xlabel('지지도 (Support)')
    plt.ylabel('신뢰도 (Confidence)')
    plt.title('신뢰도 vs 지지도')
    plt.grid(True)
    plt.tight_layout()
    
    # 향상도 분포
    plt.figure(figsize=(8, 6))
    plt.hist(rules['lift'], bins=20)
    plt.xlabel('향상도 (Lift)')
    plt.ylabel('규칙 수')
    plt.title('향상도 분포')
    plt.grid(True)
    plt.tight_layout()
    
# 분석 시각화
analyze_rules_by_metrics(rules)

# 9. 특정 카테고리 중심의 연관 규칙 찾기
def find_rules_for_category(rules, category_keyword):
    """특정 카테고리와 연관된 규칙 찾기"""
    # 선행 항목에 키워드가 포함된 규칙
    antecedent_mask = rules['antecedents'].apply(
        lambda x: any(category_keyword in item for item in x))
    
    # 후행 항목에 키워드가 포함된 규칙
    consequent_mask = rules['consequents'].apply(
        lambda x: any(category_keyword in item for item in x))
    
    # 두 조건 중 하나라도 만족하는 규칙
    return rules[antecedent_mask | consequent_mask]

# 식품 카테고리 관련 규칙 찾기
food_rules = find_rules_for_category(rules_sorted, '식품')
print("\n=== 식품 카테고리 관련 연관 규칙 ===")
print(food_rules)

# 10. 아이템 구매 빈도 분석
def analyze_item_frequency(transactions):
    """아이템 구매 빈도 분석"""
    # 모든 아이템 리스트로 펼치기
    all_items = [item for sublist in transactions for item in sublist]
    
    # 빈도수 계산
    item_counts = pd.Series(all_items).value_counts()
    
    # 시각화
    plt.figure(figsize=(10, 6))
    ax = item_counts.plot(kind='bar')
    plt.title('아이템별 구매 빈도')
    plt.xlabel('아이템')
    plt.ylabel('빈도수')
    plt.xticks(rotation=45, ha='right')
    
    # 바 위에 값 표시
    for i, v in enumerate(item_counts):
        ax.text(i, v + 0.1, str(v), ha='center')
    
    plt.tight_layout()
    
# 아이템 빈도 분석
analyze_item_frequency(transactions)

# 11. 실제 데이터에 적용할 함수
def perform_market_basket_analysis(df, item_column, min_support=0.01, min_confidence=0.3, min_lift=1.0):
    """
    장바구니 분석을 수행하는 종합 함수
    
    Parameters:
    -----------
    file_path : str
        CSV 파일 경로
    item_column : str
        아이템 목록을 포함하는 열 이름
    min_support : float, default 0.01
        최소 지지도
    min_confidence : float, default 0.3
        최소 신뢰도
    min_lift : float, default 1.0
        최소 향상도
        
    Returns:
    --------
    tuple : (frequent_itemsets, rules)
        frequent_itemsets: 빈발 아이템셋 DataFrame
        rules: 연관 규칙 DataFrame
    """
    # 1. 데이터 로드
    
    # 2. 아이템 열 전처리
    df[item_column] = df[item_column].str.split(', ')
    transactions = df[item_column].tolist()
    
    # 3. 트랜잭션 인코딩
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
    
    # 4. 빈발 아이템셋 찾기
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)
    
    # 5. 연관 규칙 생성
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    
    # 6. 향상도로 필터링 및 정렬
    rules = rules[rules['lift'] >= min_lift]
    rules_sorted = rules.sort_values('lift', ascending=False)
    
    # 7. 결과 요약
    print(f"데이터 분석 완료!")
    print(f"총 트랜잭션 수: {len(transactions)}")
    print(f"총 고유 아이템 수: {len(te.columns_)}")
    print(f"발견된 빈발 아이템셋: {len(frequent_itemsets)}")
    print(f"발견된 연관 규칙: {len(rules_sorted)}")
    
    if len(rules_sorted) > 0:
        print("\n상위 5개 연관 규칙:")
        top_5 = rules_sorted.head(5)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
        print(top_5)
        
        # 시각화
        top_rules_fig = visualize_top_rules(rules_sorted)
        top_rules_fig.savefig('top_rules.png')
        print("\n'top_rules.png' 파일에 상위 규칙 시각화가 저장되었습니다.")
    
    return frequent_itemsets, rules_sorted

# 실제 데이터 분석 시 아래 코드 사용
result = perform_market_basket_analysis(df, 'depth1_list', min_support=0.005)

print("\n실제 분석 시에는 위의 perform_market_basket_analysis 함수를 사용하세요.")

[['식품', '생활-건강'], ['식품', '패션의류'], ['식품'], ['화장품-미용', '식품'], ['식품'], ['식품', '생활-건강'], ['식품', '패션잡화'], ['가구-인테리어'], ['가구-인테리어'], ['디지털-가전'], ['생활-건강'], ['생활-건강'], ['생활-건강'], ['생활-건강'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['식품'], ['패션잡화'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['화장품-미용'], ['생활-건강', '식품', '디지털-가전'], ['화장품-미용', '식품'], ['패션의류', '패션잡화'], ['디지털-가전', '생활-건강'], ['생활-건강', '식품'], ['디지털-가전'], ['디지털-가전'], ['생활-건강'], ['생활-건강'], ['생활-건강'], ['생활-건강'], ['생활-건강'], ['스포츠-레저'], ['식품'], ['식품'], ['식품'], ['식품'], ['패션의류'], ['가구-인테리어'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['디지털-가전'], ['생활-건강'], [

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\seaborn\categorical.py:638: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  vals = pd.Series(vals, index=index)


ValueError: min() arg is an empty sequence

<Figure size 720x432 with 0 Axes>